# GroupKFold(5) CV Experiment

**CRITICAL HYPOTHESIS**: The CV-LB gap (~10x) is due to a mismatch between our Leave-One-Out CV and the evaluation procedure.

**Evidence**:
- CV-LB relationship: LB = 4.27 × CV + 0.0527 (R² = 0.967)
- Intercept (0.0527) > Target (0.0347) - current approach CANNOT reach target
- The "mixall" kernel uses GroupKFold(5) and claims "good CV-LB"

**Implementation**:
- Overwrite CV functions to use GroupKFold(5) instead of Leave-One-Out
- This is ALLOWED since the last 3 cells remain unchanged
- Use the best model: GP(0.15) + MLP(0.55) + LGBM(0.3)

**Expected outcome**:
- CV score will be HIGHER (worse) because each test fold has multiple solvents
- But the CV-LB gap should be SMALLER
- If new CV is ~0.03-0.05, this would be closer to LB (~0.088)

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import lightgbm as lgb
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

# CRITICAL: GroupKFold(5) CV functions - OVERWRITES the Leave-One-Out functions
def generate_leave_one_out_splits(X, Y):
    """Generate Group K-Fold splits across the solvents (5-fold).
    
    This OVERWRITES the original leave-one-out function to use GroupKFold(5).
    The "mixall" kernel uses this approach and claims "good CV-LB".
    """
    groups = X["SOLVENT NAME"]
    n_groups = len(groups.unique())
    n_splits = min(5, n_groups)
    
    print(f"Using GroupKFold with {n_splits} splits for {n_groups} unique solvents")
    
    gkf = GroupKFold(n_splits=n_splits)
    
    for train_idx, test_idx in gkf.split(X, Y, groups):
        yield (
            (X.iloc[train_idx], Y.iloc[train_idx]),
            (X.iloc[test_idx], Y.iloc[test_idx]),
        )

def generate_leave_one_ramp_out_splits(X, Y):
    """Generate Group K-Fold splits across the solvent ramps (5-fold).
    
    This OVERWRITES the original leave-one-ramp-out function to use GroupKFold(5).
    """
    groups = X["SOLVENT A NAME"].astype(str) + "_" + X["SOLVENT B NAME"].astype(str)
    
    n_groups = len(groups.unique())
    n_splits = min(5, n_groups)
    
    print(f"Using GroupKFold with {n_splits} splits for {n_groups} unique ramps")
    
    gkf = GroupKFold(n_splits=n_splits)
    
    for train_idx, test_idx in gkf.split(X, Y, groups):
        yield (
            (X.iloc[train_idx], Y.iloc[train_idx]),
            (X.iloc[test_idx], Y.iloc[test_idx]),
        )

print('Data loading functions defined with GroupKFold(5) CV')

Data loading functions defined with GroupKFold(5) CV


In [3]:
# Load feature lookups
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
ACS_PCA_DF = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0)

# Filter DRFP to high-variance columns
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange: {SPANGE_DF.shape}, DRFP filtered: {DRFP_FILTERED.shape}, ACS PCA: {ACS_PCA_DF.shape}')

Spange: (26, 13), DRFP filtered: (24, 122), ACS PCA: (24, 5)


In [4]:
# Full Featurizer (for MLP and LGBM) - 145 features
class FullFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED
        self.acs_pca_df = ACS_PCA_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1] + self.drfp_df.shape[1] + self.acs_pca_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            A_acs = self.acs_pca_df.loc[X["SOLVENT A NAME"]].values
            B_acs = self.acs_pca_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
                X_drfp = B_drfp * (1 - (1-pct)) + A_drfp * (1-pct)
                X_acs = B_acs * (1 - (1-pct)) + A_acs * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
                X_drfp = A_drfp * (1 - pct) + B_drfp * pct
                X_acs = A_acs * (1 - pct) + B_acs * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
            X_acs = self.acs_pca_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange, X_drfp, X_acs])
    
    def featurize_torch(self, X, flip=False):
        return torch.tensor(self.featurize(X, flip))

print(f'Full feature dimension: {FullFeaturizer().feats_dim}')

Full feature dimension: 145


In [5]:
# Simple Featurizer (for GP) - 18 features (Spange + Arrhenius kinetics)
class SimpleFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1]  # 18 features

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange])

print(f'Simple feature dimension (for GP): {SimpleFeaturizer().feats_dim}')

Simple feature dimension (for GP): 18


In [6]:
# Enhanced MLP with residual connections (handles small batches)
class EnhancedMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, output_dim=3, dropout=0.2):
        super().__init__()
        self.input_proj = nn.Linear(input_dim, hidden_dim)
        # Use LayerNorm instead of BatchNorm to handle small batches
        self.ln1 = nn.LayerNorm(hidden_dim)
        self.hidden1 = nn.Linear(hidden_dim, hidden_dim)
        self.ln2 = nn.LayerNorm(hidden_dim)
        self.hidden2 = nn.Linear(hidden_dim, hidden_dim)
        self.ln3 = nn.LayerNorm(hidden_dim)
        self.output = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        self.act = nn.GELU()
        
    def forward(self, x):
        x = self.act(self.ln1(self.input_proj(x)))
        x = self.dropout(x)
        residual = x
        x = self.act(self.ln2(self.hidden1(x)))
        x = self.dropout(x)
        x = x + residual  # Residual connection
        residual = x
        x = self.act(self.ln3(self.hidden2(x)))
        x = self.dropout(x)
        x = x + residual  # Residual connection
        return self.output(x)

print('EnhancedMLP defined with LayerNorm (handles small batches)')

EnhancedMLP defined with LayerNorm (handles small batches)


In [7]:
# GP + MLP + LGBM Ensemble (best model from exp_035)
class GPMLPLGBMEnsemble:
    def __init__(self, data='single', gp_weight=0.15, mlp_weight=0.55, lgbm_weight=0.3):
        self.data_type = data
        self.gp_weight = gp_weight
        self.mlp_weight = mlp_weight
        self.lgbm_weight = lgbm_weight
        
        # Featurizers
        self.full_featurizer = FullFeaturizer(mixed=(data == 'full'))
        self.simple_featurizer = SimpleFeaturizer(mixed=(data == 'full'))
        
        # Scalers
        self.full_scaler = StandardScaler()
        self.simple_scaler = StandardScaler()
        self.y_scaler = StandardScaler()
        
        # Models
        self.mlp = EnhancedMLP(self.full_featurizer.feats_dim, hidden_dim=128, output_dim=3, dropout=0.2).to(device)
        self.gps = []  # One GP per target
        self.lgbm_models = []  # One LGBM per target
        
    def train_model(self, X, Y):
        # Featurize
        X_full = self.full_featurizer.featurize(X)
        X_simple = self.simple_featurizer.featurize(X)
        Y_np = Y.values
        
        # Scale
        X_full_scaled = self.full_scaler.fit_transform(X_full)
        X_simple_scaled = self.simple_scaler.fit_transform(X_simple)
        Y_scaled = self.y_scaler.fit_transform(Y_np)
        
        # Train MLP
        self._train_mlp(X_full_scaled, Y_scaled)
        
        # Train GPs (one per target)
        self.gps = []
        for i in range(3):
            kernel = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5) + WhiteKernel(noise_level=0.1)
            gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=3, normalize_y=True, random_state=42)
            gp.fit(X_simple_scaled, Y_scaled[:, i])
            self.gps.append(gp)
        
        # Train LGBM (one per target)
        self.lgbm_models = []
        for i in range(3):
            lgbm_model = lgb.LGBMRegressor(
                n_estimators=200, learning_rate=0.05, max_depth=6,
                num_leaves=31, min_child_samples=10, reg_alpha=0.1, reg_lambda=0.1,
                random_state=42, verbose=-1
            )
            lgbm_model.fit(X_full_scaled, Y_scaled[:, i])
            self.lgbm_models.append(lgbm_model)
    
    def _train_mlp(self, X_scaled, Y_scaled):
        X_tensor = torch.tensor(X_scaled, dtype=torch.double).to(device)
        Y_tensor = torch.tensor(Y_scaled, dtype=torch.double).to(device)
        
        dataset = TensorDataset(X_tensor, Y_tensor)
        loader = DataLoader(dataset, batch_size=32, shuffle=True)
        
        optimizer = torch.optim.AdamW(self.mlp.parameters(), lr=1e-3, weight_decay=1e-4)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=150)
        
        # Weighted MSE loss
        target_weights = torch.tensor([1.0, 1.0, 2.0], dtype=torch.double).to(device)
        
        self.mlp.train()
        for epoch in range(150):
            for X_batch, Y_batch in loader:
                optimizer.zero_grad()
                pred = self.mlp(X_batch)
                loss = ((pred - Y_batch) ** 2 * target_weights).mean()
                loss.backward()
                optimizer.step()
            scheduler.step()
    
    def predict(self, X):
        # Featurize
        X_full = self.full_featurizer.featurize(X)
        X_simple = self.simple_featurizer.featurize(X)
        
        # Scale
        X_full_scaled = self.full_scaler.transform(X_full)
        X_simple_scaled = self.simple_scaler.transform(X_simple)
        
        # MLP prediction
        self.mlp.eval()
        with torch.no_grad():
            X_tensor = torch.tensor(X_full_scaled, dtype=torch.double).to(device)
            mlp_pred_scaled = self.mlp(X_tensor).cpu().numpy()
        
        # GP prediction
        gp_pred_scaled = np.column_stack([gp.predict(X_simple_scaled) for gp in self.gps])
        
        # LGBM prediction
        lgbm_pred_scaled = np.column_stack([lgbm.predict(X_full_scaled) for lgbm in self.lgbm_models])
        
        # Ensemble (weighted average in scaled space)
        ensemble_pred_scaled = (
            self.gp_weight * gp_pred_scaled + 
            self.mlp_weight * mlp_pred_scaled + 
            self.lgbm_weight * lgbm_pred_scaled
        )
        
        # Inverse transform
        ensemble_pred = self.y_scaler.inverse_transform(ensemble_pred_scaled)
        
        # Clip to valid range
        ensemble_pred = np.clip(ensemble_pred, 0, 1)
        
        return torch.tensor(ensemble_pred)
    
    def predict_with_tta(self, X):
        """Predict with test-time augmentation for mixed solvents."""
        if self.data_type == 'single':
            return self.predict(X)
        
        # For mixed solvents, average predictions from both orderings
        pred1 = self.predict(X)
        
        # Create flipped version
        X_flip = X.copy()
        X_flip["SOLVENT A NAME"] = X["SOLVENT B NAME"]
        X_flip["SOLVENT B NAME"] = X["SOLVENT A NAME"]
        X_flip["SolventB%"] = 1 - X["SolventB%"]
        
        # Featurize with flip=True
        X_full_flip = self.full_featurizer.featurize(X_flip, flip=False)
        X_simple_flip = self.simple_featurizer.featurize(X_flip, flip=False)
        
        X_full_scaled = self.full_scaler.transform(X_full_flip)
        X_simple_scaled = self.simple_scaler.transform(X_simple_flip)
        
        self.mlp.eval()
        with torch.no_grad():
            X_tensor = torch.tensor(X_full_scaled, dtype=torch.double).to(device)
            mlp_pred_scaled = self.mlp(X_tensor).cpu().numpy()
        
        gp_pred_scaled = np.column_stack([gp.predict(X_simple_scaled) for gp in self.gps])
        lgbm_pred_scaled = np.column_stack([lgbm.predict(X_full_scaled) for lgbm in self.lgbm_models])
        
        ensemble_pred_scaled = (
            self.gp_weight * gp_pred_scaled + 
            self.mlp_weight * mlp_pred_scaled + 
            self.lgbm_weight * lgbm_pred_scaled
        )
        
        pred2 = self.y_scaler.inverse_transform(ensemble_pred_scaled)
        pred2 = np.clip(pred2, 0, 1)
        pred2 = torch.tensor(pred2)
        
        # Average
        return (pred1 + pred2) / 2

print('GPMLPLGBMEnsemble defined with GP(0.15) + MLP(0.55) + LGBM(0.3)')

GPMLPLGBMEnsemble defined with GP(0.15) + MLP(0.55) + LGBM(0.3)


In [8]:
# Test the model on a quick sanity check
X_single, Y_single = load_data("single_solvent")
print(f"Single solvent data: {X_single.shape[0]} samples, {len(X_single['SOLVENT NAME'].unique())} unique solvents")

X_full, Y_full = load_data("full")
ramps = X_full[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
print(f"Full data: {X_full.shape[0]} samples, {len(ramps)} unique ramps")

# Test GroupKFold splits
print("\nTesting GroupKFold splits:")
for i, split in enumerate(generate_leave_one_out_splits(X_single, Y_single)):
    (train_X, train_Y), (test_X, test_Y) = split
    print(f"  Fold {i}: Train {len(train_X)}, Test {len(test_X)}, Test solvents: {test_X['SOLVENT NAME'].unique().tolist()[:3]}...")
    if i >= 2:
        print("  ...")
        break

Single solvent data: 656 samples, 24 unique solvents
Full data: 1227 samples, 13 unique ramps

Testing GroupKFold splits:
Using GroupKFold with 5 splits for 24 unique solvents
  Fold 0: Train 531, Test 125, Test solvents: ['IPA [Propan-2-ol]', 'Acetonitrile', 'Diethyl Ether [Ether]']...
  Fold 1: Train 526, Test 130, Test solvents: ['2-Methyltetrahydrofuran [2-MeTHF]', 'Cyclohexane', 'Decanol']...
  Fold 2: Train 521, Test 135, Test solvents: ['Methanol', 'Ethylene Glycol [1,2-Ethanediol]', 'Ethanol']...
  ...


In [9]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = GPMLPLGBMEnsemble(data='single')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

Using GroupKFold with 5 splits for 24 unique solvents


1it [00:15, 15.85s/it]

2it [00:30, 15.24s/it]

3it [00:47, 15.81s/it]

4it [01:02, 15.81s/it]

5it [01:16, 15.08s/it]

5it [01:16, 15.35s/it]

In [10]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = GPMLPLGBMEnsemble(data='full')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict_with_tta(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

Using GroupKFold with 5 splits for 13 unique ramps


1it [01:07, 67.50s/it]

2it [02:12, 66.27s/it]

3it [03:17, 65.68s/it]

4it [04:25, 66.25s/it]

5it [05:29, 65.62s/it]

5it [05:29, 65.90s/it]

In [11]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [12]:
# Calculate CV score with GroupKFold(5)
# NOTE: This is different from Leave-One-Out CV!

print("=== GroupKFold(5) CV SCORE ===")
print("NOTE: This uses 5 folds instead of 24/13 folds")
print("Expected: CV will be HIGHER (worse) but closer to LB")

# For GroupKFold, we need to recalculate the actuals in the same order as predictions
# The predictions are stored with fold_idx and row_idx

# Single solvent
X_single, Y_single = load_data("single_solvent")
actuals_single = []
preds_single = []

for fold_idx in sorted(submission_single_solvent['fold'].unique()):
    fold_preds = submission_single_solvent[submission_single_solvent['fold'] == fold_idx]
    preds_single.append(fold_preds[['target_1', 'target_2', 'target_3']].values)

preds_single = np.vstack(preds_single)

# Get actuals in same order (by regenerating the splits)
gkf = GroupKFold(n_splits=5)
groups = X_single["SOLVENT NAME"]
for train_idx, test_idx in gkf.split(X_single, Y_single, groups):
    actuals_single.append(Y_single.iloc[test_idx].values)
actuals_single = np.vstack(actuals_single)

# Full data
X_full, Y_full = load_data("full")
actuals_full = []
preds_full = []

for fold_idx in sorted(submission_full_data['fold'].unique()):
    fold_preds = submission_full_data[submission_full_data['fold'] == fold_idx]
    preds_full.append(fold_preds[['target_1', 'target_2', 'target_3']].values)

preds_full = np.vstack(preds_full)

# Get actuals in same order
groups_full = X_full["SOLVENT A NAME"].astype(str) + "_" + X_full["SOLVENT B NAME"].astype(str)
for train_idx, test_idx in gkf.split(X_full, Y_full, groups_full):
    actuals_full.append(Y_full.iloc[test_idx].values)
actuals_full = np.vstack(actuals_full)

# Calculate MSE
mse_single = np.mean((actuals_single - preds_single) ** 2)
mse_full = np.mean((actuals_full - preds_full) ** 2)
n_single = len(actuals_single)
n_full = len(actuals_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\nSingle Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE (GroupKFold-5): {overall_mse:.6f}')

print(f'\n=== COMPARISON ===')
print(f'exp_041 (Leave-One-Out CV): 0.008199')
print(f'exp_042 (GroupKFold-5 CV): {overall_mse:.6f}')

if overall_mse > 0.008199:
    ratio = overall_mse / 0.008199
    print(f'\nGroupKFold CV is {ratio:.2f}x higher than Leave-One-Out CV')
    print(f'This is EXPECTED - GroupKFold tests on multiple solvents per fold')
    
    # Estimate what LB might be with this CV
    # Current relationship: LB = 4.27 * CV + 0.0527
    # If GroupKFold CV is closer to LB, the relationship should be different
    estimated_lb = 4.27 * overall_mse + 0.0527
    print(f'\nEstimated LB (using old relationship): {estimated_lb:.4f}')
    print(f'Best LB so far: 0.0877')
    print(f'Target: 0.0347')

=== GroupKFold(5) CV SCORE ===
NOTE: This uses 5 folds instead of 24/13 folds
Expected: CV will be HIGHER (worse) but closer to LB

Single Solvent MSE: 0.009957 (n=656)
Full Data MSE: 0.008852 (n=1227)
Overall MSE (GroupKFold-5): 0.009237

=== COMPARISON ===
exp_041 (Leave-One-Out CV): 0.008199
exp_042 (GroupKFold-5 CV): 0.009237

GroupKFold CV is 1.13x higher than Leave-One-Out CV
This is EXPECTED - GroupKFold tests on multiple solvents per fold

Estimated LB (using old relationship): 0.0921
Best LB so far: 0.0877
Target: 0.0347
